In [1]:
import os
import csv

from BBC_scrapper_amharic import BBCScraper

In [2]:
pages = 1
csv_file_path = "data/bbc_amharic.csv"

# Replace with the base URL of the page you want to scrape
base_url = "https://www.bbc.com/amharic/topics/c7zp57r92v5t"
prefix = "https://www.bbc.com/amharic/articles"

In [39]:
# save the data to a csv file


def save_to_csv(news_articles, csv_file_path):
    # Save the articles to a csv file
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=news_articles[0].keys())
        writer.writeheader()
        writer.writerows(news_articles)

In [4]:
total_articles = []

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

for page_number in range(1, pages + 1):
    page_url = f"https://www.bbc.com/amharic/topics/c7zp57r92v5t?page={page_number}"
    print(page_url)

    scrapper = BBCScraper(page_url, prefix)
    news_articles = scrapper.get_articles()
    total_articles.extend(news_articles)
    print(news_articles)

    # Save the articles to a csv file
    save_to_csv(news_articles, csv_file_path)

https://www.bbc.com/amharic/topics/c7zp57r92v5t?page=1
[{'url': 'https://www.bbc.com/amharic/articles/cyee975r989o', 'title': 'ኬንያ በአሜሪካ የኔቶ አባል ያልሆነች አጋር መባሏ ለቀጣናው መረጋጋት ምን ፋይዳ አለው? - BBC News አማርኛ', 'date': None, 'image_url': 'https://a1.api.bbc.co.uk/hit.xiti?s=598342&s2=4&p=amharic.articles.cyee975r989o.page&x1=[urn%3Abbc%3Aoptimo%3Aasset%3Acyee975r989o]&x2=[responsive]&x3=[news-amharic]&x4=[am]&x7=[article]&x8=[simorgh-nojs]&x9=[%E1%8A%AC%E1%8A%95%E1%8B%AB%2520%E1%89%A0%E1%8A%A0%E1%88%9C%E1%88%AA%E1%8A%AB%2520%E1%8B%A8%E1%8A%94%E1%89%B6%2520%E1%8A%A0%E1%89%A3%E1%88%8D%2520%E1%8B%AB%E1%88%8D%E1%88%86%E1%8A%90%E1%89%BD%2520%E1%8A%A0%E1%8C%8B%E1%88%AD%2520%E1%88%98%E1%89%A3%E1%88%8F%2520%E1%88%88%E1%89%80%E1%8C%A3%E1%8A%93%E1%8B%8D%2520%E1%88%98%E1%88%A8%E1%8C%8B%E1%8C%8B%E1%89%B5%2520%E1%88%9D%E1%8A%95%2520%E1%8D%8B%E1%8B%AD%E1%8B%B3%2520%E1%8A%A0%E1%88%88%E1%8B%8D%3F]&x11=[2024-05-25T04%3A50%3A14.442Z]&x12=[2024-05-25T04%3A50%3A14.442Z]&x13=[Kenya~United%2BStates~United%2BNations~S

In [5]:
total_articles[0]

{'url': 'https://www.bbc.com/amharic/articles/cyee975r989o',
 'title': 'ኬንያ በአሜሪካ የኔቶ አባል ያልሆነች አጋር መባሏ ለቀጣናው መረጋጋት ምን ፋይዳ አለው? - BBC News አማርኛ',
 'date': None,
 'image_url': 'https://a1.api.bbc.co.uk/hit.xiti?s=598342&s2=4&p=amharic.articles.cyee975r989o.page&x1=[urn%3Abbc%3Aoptimo%3Aasset%3Acyee975r989o]&x2=[responsive]&x3=[news-amharic]&x4=[am]&x7=[article]&x8=[simorgh-nojs]&x9=[%E1%8A%AC%E1%8A%95%E1%8B%AB%2520%E1%89%A0%E1%8A%A0%E1%88%9C%E1%88%AA%E1%8A%AB%2520%E1%8B%A8%E1%8A%94%E1%89%B6%2520%E1%8A%A0%E1%89%A3%E1%88%8D%2520%E1%8B%AB%E1%88%8D%E1%88%86%E1%8A%90%E1%89%BD%2520%E1%8A%A0%E1%8C%8B%E1%88%AD%2520%E1%88%98%E1%89%A3%E1%88%8F%2520%E1%88%88%E1%89%80%E1%8C%A3%E1%8A%93%E1%8B%8D%2520%E1%88%98%E1%88%A8%E1%8C%8B%E1%8C%8B%E1%89%B5%2520%E1%88%9D%E1%8A%95%2520%E1%8D%8B%E1%8B%AD%E1%8B%B3%2520%E1%8A%A0%E1%88%88%E1%8B%8D%3F]&x11=[2024-05-25T04%3A50%3A14.442Z]&x12=[2024-05-25T04%3A50%3A14.442Z]&x13=[Kenya~United%2BStates~United%2BNations~Somalia~Military~Africa~Ethiopia]&x14=[7d7c35fa-f724-4

### Use kafka to store the data and use faust to process the data

In [25]:
from kafka import KafkaProducer
import json

In [26]:
# Create a Kafka producer
producer = KafkaProducer(bootstrap_servers='localhost:9092',
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [33]:
# models.py
import faust

'''
- This is a faust record that will be used to store the scraped data
- Faust record, which is a type of data model that Faust can use to understand the structure of your messages.
'''
class ScrapedData(faust.Record, serializer='json'):
    url: str
    title: str
    source: str

    def to_dict(self):
        return {
            'url': self.url,
            'title': self.title,
            'source': self.source,
        }

In [34]:
# create a scrapedData object
scraped_data = ScrapedData(url=total_articles[0]['url'], title=total_articles[0]['title'], source=total_articles[0]['source'])

In [38]:
# send the scraped data to the 'scraped_data' Kafka topic
producer.send('scraped_data', value=scraped_data.to_dict())